In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

In [14]:
from pathlib import Path
import pandas as pd

# Find the processed directory automatically
CANDIDATES = [
    Path.cwd() / "data" / "processed",
    Path.home() / "Documents" / "ADS-505-Final-Project-Pharma-Sales-Forecast" / "data" / "processed",
    Path(r"C:\Users\kiara\Documents\ADS-505-Final-Project-Pharma-Sales-Forecast\data\processed"),
]

def pick_processed_dir(candidates):
    for d in candidates:
        if d.exists() and any(d.glob("sales*clean*")):
            print(f"Using processed dir: {d}")
            return d
    raise FileNotFoundError("Could not locate a 'data/processed' folder containing sales*clean files.")

PROCESSED = pick_processed_dir(CANDIDATES)

# Resolve filenames regardless of extension / slight name differences
def resolve_file(stem: str) -> Path | None:
    # Try exact common extensions first
    for ext in (".csv", ".xlsx", ".CSV", ".XLSX"):
        p = PROCESSED / f"{stem}{ext}"
        if p.exists():
            return p
    # Fallback: fuzzy match (e.g., 'salesdaily_clean*')
    hits = sorted(PROCESSED.glob(f"{stem}*"))
    return hits[0] if hits else None

files = {
    "daily":   resolve_file("salesdaily_clean"),
    "weekly":  resolve_file("salesweekly_clean"),
    "monthly": resolve_file("salesmonthly_clean"),
    "hourly":  resolve_file("saleshourly_clean"),
}

for k, fp in files.items():
    print(f"{k:<7} ->", fp if fp else "NOT FOUND")

# 3) Helper to read CSV or Excel uniformly
def read_any(fp: Path, parse_dates=("datum",)):
    if fp.suffix.lower() == ".csv":
        return pd.read_csv(fp, parse_dates=list(parse_dates))
    elif fp.suffix.lower() in {".xlsx", ".xls"}:
        return pd.read_excel(fp, parse_dates=list(parse_dates))
    else:
        raise ValueError(f"Unsupported file type: {fp.suffix}")

# Example loads 
daily   = read_any(files["daily"])   if files["daily"]   else None
weekly  = read_any(files["weekly"])  if files["weekly"]  else None
monthly = read_any(files["monthly"]) if files["monthly"] else None
hourly  = read_any(files["hourly"])  if files["hourly"]  else None

for name, df in [("daily", daily), ("weekly", weekly), ("monthly", monthly), ("hourly", hourly)]:
    if df is not None:
        print(f"{name}: {df.shape} | cols: {list(df.columns)[:6]} ...")


Using processed dir: C:\Users\kiara\Documents\ADS-505-Final-Project-Pharma-Sales-Forecast\data\processed
daily   -> C:\Users\kiara\Documents\ADS-505-Final-Project-Pharma-Sales-Forecast\data\processed\salesdaily_clean.csv
weekly  -> C:\Users\kiara\Documents\ADS-505-Final-Project-Pharma-Sales-Forecast\data\processed\salesweekly_clean.csv
monthly -> C:\Users\kiara\Documents\ADS-505-Final-Project-Pharma-Sales-Forecast\data\processed\salesmonthly_clean.csv
hourly  -> C:\Users\kiara\Documents\ADS-505-Final-Project-Pharma-Sales-Forecast\data\processed\saleshourly_clean.csv
daily: (2106, 13) | cols: ['datum', 'M01AB', 'M01AE', 'N02BA', 'N02BE', 'N05B'] ...
weekly: (302, 9) | cols: ['datum', 'M01AB', 'M01AE', 'N02BA', 'N02BE', 'N05B'] ...
monthly: (70, 9) | cols: ['datum', 'M01AB', 'M01AE', 'N02BA', 'N02BE', 'N05B'] ...
hourly: (50532, 13) | cols: ['datum', 'M01AB', 'M01AE', 'N02BA', 'N02BE', 'N05B'] ...


In [15]:
# Work on daily sales data
df = daily.copy()

# Convert date column
df['datum'] = pd.to_datetime(df['datum'])
df = df.sort_values('datum')

# Create temporal features
df['year'] = df['datum'].dt.year
df['month'] = df['datum'].dt.month
df['day'] = df['datum'].dt.day
df['dayofweek'] = df['datum'].dt.dayofweek
df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(int)

# Aggregate total sales
df['total_sales'] = df[['M01AB', 'M01AE', 'N02BA', 'N02BE', 'N05B', 'N05C', 'R03', 'R06']].sum(axis=1)

# Lag and rolling features (example)
for lag in [1, 7, 30]:
    df[f'total_sales_lag_{lag}'] = df['total_sales'].shift(lag)

df['rolling_mean_7'] = df['total_sales'].shift(1).rolling(window=7).mean()
df['rolling_std_7'] = df['total_sales'].shift(1).rolling(window=7).std()

# Drop missing rows created by shifts
df = df.dropna().reset_index(drop=True)

print(df.shape)
df.head()

(2076, 24)


,datum,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06,Year,...,month,day,dayofweek,is_weekend,total_sales,total_sales_lag_1,total_sales_lag_7,total_sales_lag_30,rolling_mean_7,rolling_std_7
0,2014-02-01,4.33,4.32,5.0,43.0,13.0,1.0,14.0,0.0,2014,...,2,1,5,1,84.65,47.68,81.80,48.47,51.915714,15.821377
1,2014-02-02,7.00,3.00,0.2,13.5,6.0,2.0,8.0,0.0,2014,...,2,2,6,1,39.70,84.65,34.01,107.00,52.322857,16.729208
2,2014-02-03,5.00,1.00,8.5,32.4,16.0,1.0,1.0,0.0,2014,...,2,3,0,0,64.90,39.70,56.81,91.35,53.135714,15.803739
3,2014-02-04,1.33,3.00,7.0,30.6,8.0,1.0,17.0,2.0,2014,...,2,4,1,0,69.93,64.90,58.52,66.10,54.291429,16.401714
4,2014-02-05,3.00,4.02,6.2,32.4,15.0,1.0,1.0,1.0,2014,...,2,5,2,0,63.62,69.93,39.33,58.20,55.921429,17.426910


In [16]:
# Time-based split
cutoff = '2019-01-01'
train = df[df['datum'] < cutoff].copy()
test = df[df['datum'] >= cutoff].copy()

print(f"Train: {train.shape}, Test: {test.shape}")

Train: (1795, 24), Test: (281, 24)


In [19]:
from pathlib import Path

# Reuse the processed dir from the files you already loaded
# (files['daily'] is the full path to salesdaily_clean.csv)
out_dir = Path(files['daily']).parent   # -> ...\ADS-505-Final-Project-Pharma-Sales-Forecast\data\processed
out_dir.mkdir(parents=True, exist_ok=True)

print("Saving to:", out_dir)

# save
train.to_csv(out_dir / "salesdaily_train_fe.csv", index=False)
test.to_csv(out_dir / "salesdaily_test_fe.csv", index=False)
print(" Feature-engineered train/test saved.")


Saving to: C:\Users\kiara\Documents\ADS-505-Final-Project-Pharma-Sales-Forecast\data\processed
 Feature-engineered train/test saved.


In [17]:
out_dir = Path.cwd() / "data" / "processed"
train.to_csv(out_dir / "salesdaily_train_fe.csv", index=False)
test.to_csv(out_dir / "salesdaily_test_fe.csv", index=False)
print("Feature-engineered train/test saved ")

OSError: Cannot save file into a non-existent directory: 'c:\Users\kiara\Downloads\data\processed'